In [21]:
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree

def compute_metrics(gt_points, gen_points):
    """
    Computes various metrics to evaluate the accuracy of a generated point cloud against the ground truth.
    Handles cases where the point counts differ.
    
    Args:
        gt_points (numpy.ndarray): Ground truth points, shape (N1, 3).
        gen_points (numpy.ndarray): Generated points, shape (N2, 3).
        
    Returns:
        dict: A dictionary containing the computed metrics.
    """
    # Compute pairwise distances using k-D tree for efficiency
    gt_kdtree = cKDTree(gt_points)
    gen_kdtree = cKDTree(gen_points)


    gt_set = set(map(tuple, gt_points))
    gen_set = set(map(tuple, gen_points))
    
    # For Chamfer Distance and Hausdorff Distance
    gen_to_gt_distances, _ = gen_kdtree.query(gt_points, k=1)
    gt_to_gen_distances, _ = gt_kdtree.query(gen_points, k=1)
    
    
    # Metrics
    # Chamfer Distance (First Term)
    chamfer_distance = np.mean(gen_to_gt_distances ** 2) + np.mean(gt_to_gen_distances ** 2)
    
    # Hausdorff Distance
    hausdorff_distance = max(np.max(gen_to_gt_distances), np.max(gt_to_gen_distances))
    
    # MSE and RMSE
    mse = np.mean(gen_to_gt_distances ** 2)  # Only for nearest points
    rmse = np.sqrt(mse)
    
    # MAE
    mae = np.mean(gen_to_gt_distances)  # Absolute distance
    
    # Signal-to-Noise Ratio (SNR)
    signal_power = np.sum(np.linalg.norm(gt_points, axis=1) ** 2)
    noise_power = np.sum(gen_to_gt_distances ** 2)
    snr = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')

    # Exact matches
    exact_matches = gt_set & gen_set
    matched_count = len(exact_matches)
    
    # Missing and extra points
    missing_count = len(gt_set - gen_set)
    extra_count = len(gen_set - gt_set)
    
    # Completeness: Fraction of GT points matched
    completeness = matched_count / len(gt_set) if len(gt_set) > 0 else 1.0
    
    # Accuracy: Fraction of Gen points that match GT
    accuracy = matched_count / len(gen_set) if len(gen_set) > 0 else 1.0
    
        
    return {
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "Chamfer Distance": chamfer_distance,
        "Hausdorff Distance": hausdorff_distance,
        "SNR": snr,
        "Exact Matches": matched_count,
        "Missing Points": missing_count,
        "Extra Points": extra_count,
        "Completeness": completeness,
        "Accuracy": accuracy,
        "Chamfer Distance": chamfer_distance
    }


def ply_metrics(gt_ply_path, gen_ply_path):
    """
    Computes metrics for a pair of point clouds given their file paths.
    
    Args:
        gt_ply_path (str): Path to the ground truth point cloud.
        gen_ply_path (str): Path to the generated point cloud.
        
    Returns:
        dict: A dictionary containing the computed metrics.
    """
    # Load point clouds
    gt_pcd = o3d.io.read_point_cloud(gt_ply_path)
    gen_pcd = o3d.io.read_point_cloud(gen_ply_path)
    
    # Convert to numpy arrays
    gt_points = np.asarray(gt_pcd.points)
    gen_points = np.asarray(gen_pcd.points)
    
    metrics = compute_metrics(gt_points, gen_points)
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    return metrics

In [27]:
bicycle_metrics = ply_metrics("data/points_bicycle_gt.ply", "output/bicycle/cluster/bicycle.ply")

MSE: 1.2573330356184421e-05
RMSE: 0.003545889219389746
MAE: 0.00021460280284468124
Chamfer Distance: 0.004557185225284462
Hausdorff Distance: 0.4778152120945047
SNR: 50.70297969988866
Exact Matches: 6308
Missing Points: 26
Extra Points: 689
Completeness: 0.9958951689295864
Accuracy: 0.9015292268114906


In [23]:
bonsai_metrics = ply_metrics("data/points_bonsai_gt.ply", "output/bonsai/cluster/bonsai.ply")


MSE: 0.0001709805336266227
RMSE: 0.013075952494048864
MAE: 0.001834674022331613
Chamfer Distance: 0.00023835957867605645
Hausdorff Distance: 0.2453098323343594
SNR: 43.172962135626975
Exact Matches: 24531
Missing Points: 689
Extra Points: 135
Completeness: 0.972680412371134
Accuracy: 0.9945268791048407


In [28]:
kitchen_metrics = ply_metrics("data/points_kitchen_gt.ply", "output/kitchen/cluster/kitchen.ply")


MSE: 0.0010305423646034932
RMSE: 0.03210206168774045
MAE: 0.01045092367960673
Chamfer Distance: 0.0012296781090043564
Hausdorff Distance: 0.4269672716415563
SNR: 36.37818919929256
Exact Matches: 32827
Missing Points: 4811
Extra Points: 767
Completeness: 0.8721770551038843
Accuracy: 0.9771685420015479


In [25]:
garden_metrics = ply_metrics("data/points_garden_gt.ply", "output/garden/cluster/garden.ply")


MSE: 0.0011595296083539836
RMSE: 0.034051866444498806
MAE: 0.009683864530969196
Chamfer Distance: 0.0012840190732325805
Hausdorff Distance: 0.5481564818660584
SNR: 36.45773451258344
Exact Matches: 12733
Missing Points: 1322
Extra Points: 38
Completeness: 0.9059409462824618
Accuracy: 0.9970245086524157
